# Lab: Bring your own script with Amazon SageMaker

## Sklearn script mode training and serving
Script mode is a training script format for a number of supported frameworks that lets you execute the training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native SKlearn support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). In this example, we will show how easily you can train a SageMaker using scikit-learn and with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker SKlearn container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-scikit-learn-spark.html). 

## Set up the environment
Let's start by setting up the environment:

In [ ]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import os
import tarfile
import pandas as pd

In [ ]:
import sagemaker

#define samemaker role 

#define sagemaker session

#define default bucket

## Training data
we download the Iris data from UCI Machine Learning repository directly from the web.

In [ ]:
#download the data from internet



## Train and test split
We split the data into train and test set

In [ ]:
#read in the data with the headers



Load our data to our S3 ready for training using our script.

In [ ]:
#import aws boto3 library

#upload the data on to your sagemaker defulat S3 bucket in a folder called training and your data called 'data.csv'


## Construct a script for brining your own SKlearn script to SageMaker
Your Scikit-learn training script must be a Python 3.6 compatible source file.
The training script is similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables. For example:
- SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
- SM_OUTPUT_DATA_DIR: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.
- Supposing two input channels, ‘train’ and ‘test’, were used in the call to the Scikit-learn estimator’s fit() method, the following will be set, following the format “SM_CHANNEL_[channel_name]”:
- SM_CHANNEL_TRAIN: A string representing the path to the directory containing data in the ‘train’ channel
- SM_CHANNEL_TEST: Same as above, but for the ‘test’ channel.

In order to save your trained Scikit-learn model for deployment on SageMaker, your training script should save your model to a certain filesystem path called model_dir. This value is accessible through the environment variable SM_MODEL_DIR.

Load the model: before a model can be served, it must be loaded. The SageMaker Scikit-learn model server loads your model by invoking a 'model_fn' function that you must provide in your script

Serve a Model: after the SageMaker model server has loaded your model by calling model_fn, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker InvokeEndpoint API calls. The SageMaker Scikit-learn model server breaks request handling into three steps:

-input processing,
-prediction, and
-output processing.

Here is the entire script and you can see all these details explained above in the script.

In [ ]:
# define your estimator using SKlearn framework



# Calling `fit`
To start a training job, we call `estimator.fit(training_data_uri)`.

An S3 location is used here as the input. fit creates a default channel named 'training', which points to this S3 location. In the training script we can then access the training data from the location stored in SM_CHANNEL_TRAINING. fit accepts a couple other types of input as well. See the API doc [here](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) for details.

When training starts, the Scikit-learn container executes BYO_sklearn_main.py, passing hyperparameters and model_dir from the estimator as script arguments. Because we didn't define either in this example, no hyperparameters are passed, and model_dir defaults to `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`, so the script execution is as follows:

`BYO_sklearn_main.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`

When training is complete, the training job will upload the saved model to S3 for deployment.

In [ ]:
#call the fit function and pass on your data you  uploaded to S3 above for the training to start


## Deploy
We are now ready to deploy our model to Sagemaker hosting services and make real time predictions

In [ ]:
#deploy the model to a real time endpoint


Let's now send some data to our model to predict- the data shouldbe sent in the accepted format and the code below just does that.

In [ ]:
#now get some test data to test your model


In [ ]:
#the body that you send to your model enpoint should be text/csv format, get your data to the right format before sending it to you model endpoint for prediciton


In [ ]:
#now envoke your endpoint and get predictions
